# Theoretical fuel burn per route 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
from matplotlib import colors
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from cmocean import cm as cmo
from cmcrameri import cm as scm
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection

In [2]:
pd.set_option('display.max_rows', None)

In [24]:
pd.reset_option('display.max_rows')

In [3]:
# load data
raw = pd.read_csv("../../raw/emissionV22_2025-02-24.csv")

## 1. List of unique airlines

In [25]:
airlines = raw["airlineiatacode"].unique()
airlines

array(['U2', 'SK', 'BT', 'Y4', 'VB', 'SN', 'ME', 'EK', 'WG', 'AW', 'ET',
       'VW', 'WF', 'UA', '2J', 'HF', 'QR', 'AM', 'KL', 'TS', 'MS', 'KQ',
       'KP', 'TK', 'SA', 'BA', 'AF', 'AT', 'DL', 'OP', 'TP', 'IB', 'WB',
       'P4', 'VY', 'LS', 'UX', 'TB', 'FR', 'KN', '8L', 'GY', 'MU', '6B',
       'EW', 'HV', 'OR', 'V7', 'BY', 'XR', 'AY', 'NT', 'TO', 'WK', 'DE',
       'DK', 'X3', 'EI', '4Y', 'LG', 'NO', 'W6', 'RK', 'DY', 'PM', 'QS',
       'EU', 'G5', 'GS', 'CZ', 'B6', 'XP', 'XC', 'XQ', 'GJ', 'PE', 'AA',
       'IR', 'SY', '3U', 'AQ', 'NK', 'PC', 'IO', 'W5', 'A3', 'ZF', 'JU',
       'FZ', 'J2', 'KU', 'SV', 'IY', 'DJ', 'J4', 'RJ', 'AS', 'NZ', 'VA',
       'QF', 'JQ', 'P5', 'LA', 'HU', 'VH', 'JD', 'ZL', 'MH', 'CX', 'SQ',
       'QQ', 'ZP', 'FJ', 'AR', 'FO', 'WJ', 'EP', 'CM', 'AV', 'Q4', 'G3',
       'U6', 'Y7', 'N4', 'EO', 'A4', 'SU', 'RT', '5N', '3F', 'G6', 'S7',
       'JA', 'H2', 'D2', 'WZ', 'I8', 'R3', 'UT', 'HY', 'B2', 'YC', '4G',
       'DV', 'D8', 'AD', 'FI', '3O', 'W9', 'TF', 'G

## 2. Create list of aircrafts, IATA code and let Claude fill in average fuel burn

In [26]:
dfa = pd.DataFrame(columns=["IATA code", "Aircraft name", "Average Fuel Burn [kg/hour]"])

aircrafts = raw["aircraftiatacode"].unique()
for aircraft in aircrafts:
    name = raw[raw["aircraftiatacode"] == aircraft]["aircraftname"].iloc[0]
    
    new_row = pd.DataFrame({"IATA code": [aircraft], 
                            "Aircraft name": [name],
                            "Average Fuel Burn [kg/hour]": [''],
                            })
    dfa = pd.concat([dfa, new_row], ignore_index=True)

# don't overwrite file that Claude already filled in with fuel burn values
# dfa.sort_values(by='IATA code').to_csv("aircraft_fuel_burn.csv", index=False)

In [27]:
df_fuel_burn = pd.read_csv("../open_data/aircraft_fuel_burn.csv")
df_fuel_burn

,IATA code,Aircraft name,Average Fuel Burn [kg/hour]
0,100,Fokker 100,2000.0
1,142,BAe 146-200 Passenger,2200.0
2,146,BAe 146 Passenger,2500.0
3,221,Airbus A220-100 Passenger,1700.0
4,223,Airbus A220-300 Passenger,1800.0
...,...,...,...
111,S20,Saab 2000,700.0
112,SF3,Saab 340,600.0
113,SFB,Saab 340B,620.0
114,SU9,Sukhoi Superjet 100-95,1800.0


# 3. Outermost loop over airlines

In [28]:
new = pd.DataFrame()

for airline in airlines:
    flights = raw[raw["airlineiatacode"] == airline]

    # unique list of departures within airline
    departures = flights["emissionflightinfo_departureiatacode"].unique()

    for departure in departures:
        flights_dep = flights[flights["emissionflightinfo_departureiatacode"] == departure]

        # unique list of arrivals within departures
        arrivals = flights_dep["emissionflightinfo_arrivaliatacode"].unique()

        for arrival in arrivals:
            
            aircrafts = flights_dep[flights_dep["emissionflightinfo_arrivaliatacode"] == arrival]
            n_aircrafts = len(aircrafts)

            total_fuel_burn = 0
            total_seats = 0

            for aircraft, seats, frequency in zip(aircrafts["aircraftiatacode"], aircrafts["seatcount"], aircrafts["frequency"]):
            
                fuel_burn = np.float32(df_fuel_burn[df_fuel_burn['IATA code'] == aircraft]["Average Fuel Burn [kg/hour]"].iloc[0])
                total_fuel_burn += fuel_burn*frequency
                total_seats += seats*frequency

            fuel_burn_per_seat = total_fuel_burn/total_seats
            
            aircrafts_str = '; '.join([aircraft for aircraft in aircrafts["aircraftiatacode"]])
            
            new_row = pd.DataFrame({"Airline": [airline], 
                                    "Departure": [departure],
                                    "Arrival": [arrival],
                                    "Aircrafts": [aircrafts_str],
                                    "Fuel burn [kg/hour/seat]": [fuel_burn_per_seat],
                                   })
            new = pd.concat([new, new_row], ignore_index=True)

new.to_csv("../data/by_route_fuel_burn.csv", index=False)

In [32]:
new

,Airline,Departure,Arrival,Aircrafts,Fuel burn [kg/hour/seat]
0,U2,ABZ,LTN,319; 320,15.542035
1,U2,ABZ,GVA,320,13.978495
2,U2,ABZ,LGW,319; 320,15.872722
3,U2,ACE,BER,320,14.207650
4,U2,ACE,AMS,320,13.978495
...,...,...,...,...,...
81839,8T,YZF,YSG,DH2,26.315790
81840,N5,YMX,YVO,737,20.168068
81841,N5,YVO,YMX,737,20.168068
81842,9M,YVR,YYD,DH1,12.162162
